In [1]:
import os
import shutil
import subprocess

import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
from scipy.sparse import csr_matrix

In [2]:
# Clone repo at https://github.com/LieberInstitute/HumanPilot and set DIR to it
DIR = "../HumanPilot"
OUT_DIR = "data/dlpfc/raw/spatialLIBD"

In [3]:
aws_sources_df = pd.read_csv(os.path.join(DIR, "AWS_File_locations.tsv"), sep="\t", index_col=0)
aws_sources_df

,h5_filtered,h5_raw,image_full,image_hi,image_lo,loupe
SampleID,,,,,,
151507,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...
151508,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...
151509,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...
151510,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...
151669,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...
151670,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...
151671,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...
151672,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...
151673,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...,https://spatial-dlpfc.s3.us-east-2.amazonaws.c...


In [4]:
# if not os.path.exists(OUT_DIR):
#     os.makedirs(OUT_DIR)


for sample_id in aws_sources_df.index:

    sdir = os.path.join(OUT_DIR, str(sample_id))
    sdir_sp = os.path.join(sdir, "spatial")

    if not os.path.exists(sdir_sp):
        os.makedirs(sdir_sp)

    shutil.copytree(
        os.path.join(DIR, "10X", str(sample_id)),
        os.path.join(sdir_sp),
        dirs_exist_ok=True,
    )

    os.rename(
        os.path.join(sdir_sp, "tissue_positions_list.txt"),
        os.path.join(sdir_sp, "tissue_positions_list.csv"),
    )

    subprocess.run(
        [
            "curl",
            "-L",
            "-o",
            "filtered_feature_bc_matrix.h5",
            aws_sources_df.loc[sample_id, "h5_filtered"],
        ],
        cwd=sdir,
    )

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9999k  100 9999k    0     0  4854k      0  0:00:02  0:00:02 --:--:-- 4854k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9036k  100 9036k    0     0  7606k      0  0:00:01  0:00:01 --:--:-- 7606k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.9M  100 10.9M    0     0  12.3M      0 --:--:-- --:--:-- --:--:-- 12.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.2M  100 10.2M    0     0  7330k      0  0:00:01  0:00:01 --:--:-- 7325k
  % Total    % Received % Xferd  Average Speed   Tim

In [5]:
adatas = []

for sample_id in aws_sources_df.index:

    sdir = os.path.join(OUT_DIR, str(sample_id))
    adatas.append(sc.read_visium(sdir))
    adatas[-1].var_names.name = "gene_names"
    adatas[-1].var = adatas[-1].var.reset_index().set_index("gene_ids")

adata = ad.concat(
    adatas,
    keys=aws_sources_df.index.astype(str),
    label="sample_id",
    index_unique=".",
    uns_merge="unique",
)

all_var = [x.var for x in adatas]
# concatenate them
all_var = pd.concat(all_var, join="outer")
# remove duplicates
all_var = all_var[~all_var.index.duplicated()]

adata.var = all_var.loc[adata.var_names]

/home/wma/miniconda3/envs/agreda/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/wma/miniconda3/envs/agreda/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/wma/miniconda3/envs/agreda/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/wma/miniconda3/envs/agreda/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/wma/miniconda3/envs/agreda/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarn

In [6]:
# for sample_id, adata_sample in zip(aws_sources_df.index, adatas):
#     adata_sample.X = csr_matrix(adata_sample.X)
#     adata_sample.write_h5ad(os.path.join(OUT_DIR, f"spatialLIBD_{sample_id}.h5ad"))

In [7]:

adata.X = csr_matrix(adata.X)


In [8]:
adata

AnnData object with n_obs × n_vars = 47681 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'sample_id'
    var: 'gene_names', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [9]:
adata_r = sc.read_h5ad(os.path.join("data/dlpfc/spatialLIBD_data", "spatialLIBD.h5ad"))

In [10]:
adata_r

AnnData object with n_obs × n_vars = 47681 × 33538
    obs: 'sample_id', 'spot', 'X', 'Y', 'Cluster', 'sum_umi', 'sum_gene', 'subject', 'position', 'replicate', 'subject_position', 'discard', 'key', 'cell_count', 'SNN_k50_k4', 'SNN_k50_k5', 'SNN_k50_k6', 'SNN_k50_k7', 'SNN_k50_k8', 'SNN_k50_k9', 'SNN_k50_k10', 'SNN_k50_k11', 'SNN_k50_k12', 'SNN_k50_k13', 'SNN_k50_k14', 'SNN_k50_k15', 'SNN_k50_k16', 'SNN_k50_k17', 'SNN_k50_k18', 'SNN_k50_k19', 'SNN_k50_k20', 'SNN_k50_k21', 'SNN_k50_k22', 'SNN_k50_k23', 'SNN_k50_k24', 'SNN_k50_k25', 'SNN_k50_k26', 'SNN_k50_k27', 'SNN_k50_k28', 'GraphBased', 'Maynard', 'Martinowich', 'layer_guess', 'layer_guess_reordered', 'layer_guess_reordered_short', 'expr_chrM', 'expr_chrM_ratio', 'SpatialDE_PCA', 'SpatialDE_pool_PCA', 'HVG_PCA', 'pseudobulk_PCA', 'markers_PCA', 'SpatialDE_UMAP', 'SpatialDE_pool_UMAP', 'HVG_UMAP', 'pseudobulk_UMAP', 'markers_UMAP', 'SpatialDE_PCA_spatial', 'SpatialDE_pool_PCA_spatial', 'HVG_PCA_spatial', 'pseudobulk_PCA_spatial', 'mar

In [11]:
adata

AnnData object with n_obs × n_vars = 47681 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'sample_id'
    var: 'gene_names', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [12]:
adata_r.obs_names.to_series().str.split(".").str[0].reset_index(drop=True).equals(adata.obs_names.to_series().str.split(".").str[0].reset_index(drop=True))

True

In [13]:
adata_r.obs.sample_id.reset_index(drop=True).equals(adata.obs.sample_id.reset_index(drop=True))

True

In [15]:
np.array_equal(adata.X.toarray(), adata_r.X.toarray())


True

In [18]:
adata.var["gene_names"].equals(adata_r.var["gene_name"])

True

In [19]:
adata.var

,gene_names,feature_types,genome
gene_ids,,,
ENSG00000243485,MIR1302-2HG,Gene Expression,GRCh38
ENSG00000237613,FAM138A,Gene Expression,GRCh38
ENSG00000186092,OR4F5,Gene Expression,GRCh38
ENSG00000238009,AL627309.1,Gene Expression,GRCh38
ENSG00000239945,AL627309.3,Gene Expression,GRCh38
...,...,...,...
ENSG00000277856,AC233755.2,Gene Expression,GRCh38
ENSG00000275063,AC233755.1,Gene Expression,GRCh38
ENSG00000271254,AC240274.1,Gene Expression,GRCh38


In [21]:
adata.var = pd.concat([adata.var, adata_r.var.drop("gene_name", axis=1)], axis=1)

In [23]:
adata_r.obs.index = adata.obs.index

In [31]:
adata_r.obs[adata.obs.columns].astype(int).equals(adata.obs.astype(int))

True

In [32]:
adata.obs = adata_r.obs

In [34]:
adata.varm = adata_r.varm

In [39]:
np.array_equal(adata.obsm["spatial"], adata.obs[["X", "Y"]].values)

True

In [35]:

adata.write_h5ad(os.path.join(OUT_DIR, "spatialLIBD.h5ad"))